In [1]:

!pip install ultralytics > /dev/null 2>&1

In [2]:
import cv2


In [3]:
import os
print(os.path.exists('/kaggle/input/input-video-1x/input_video.mp4'))
from ultralytics import YOLO 

True
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import numpy as np

# Updated detection updation on frame and others


In [5]:
model = YOLO('/kaggle/input/updated-train-result-best2/best (2).pt')

In [7]:


def count_and_track_sports_ball_detections(video_paths, output_path, initial_conf=0.1, iou_thresh=0.2, resize_factor=1.5):
    ball_detection_counts = {}
    video_stats = {}  # Dictionary to store FPS, total frames, detection frames, and ratio for each video
    detection_count=[]
    conf=[]
    # Iterate over each video path
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * resize_factor)
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * resize_factor)
        fps = cap.get(cv2.CAP_PROP_FPS)  # Get FPS (Frames Per Second)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames in the video

        # Video writer to save the output video with tracking
        video_name = video_path.split('/')[-1]
        output_video = cv2.VideoWriter(f"{output_path}/{video_name}_tracked.avi", 
                                       cv2.VideoWriter_fourcc(*'XVID'), fps, 
                                       (frame_width, frame_height))

        ball_count = 0  # Counter for sports ball detections
        detection_frames = 0  # Counter for frames with at least one detection
        no_detection_streak = 0  # Counter for consecutive frames without detections

        # Process the video frame by frame with tracking enabled
        results = model.track(video_path, conf=initial_conf, iou=iou_thresh, save=True, verbose=False)

        # Iterate over frames in the tracking result
        for frame_result in results:
            detections_found = False  # Flag to indicate detection in this frame
            frame = frame_result.orig_img  # Original frame from the result
            
            # Resize frame if a resize factor is specified
            if resize_factor != 1.0:
                frame = cv2.resize(frame, (frame_width, frame_height))

            # Iterate over detected objects in each frame
            for obj in frame_result.boxes:
                # Check if the label is 'tennis ball'
                if 'tennis ball' in frame_result.names[int(obj.cls)]:
                    ball_count += 1
                    detections_found = True  # Detection found in this frame
                    no_detection_streak = 0  # Reset streak if detection is found

                    # Get bounding box coordinates and draw on the frame
                    x1, y1, x2, y2 = map(int, obj.xyxy[0].cpu().numpy())
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, 'Tennis Ball', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    break  # Exit after first detection
            detection_count.append(detections_found)
            conf.append(initial_conf)
            
            # Increase detection frame count if any detection is found in this frame
            if detections_found:
                detection_frames += 1
            else:
                no_detection_streak += 1
            if no_detection_streak >= 5:  # Lower confidence threshold after 5 frames without detections
                initial_conf = max(0.05, initial_conf - 0.01)
            else:
                initial_conf = max(0.1, initial_conf)  # Reset to the initial confidence threshold
            
            # Write the processed frame with bounding boxes to the output video
            output_video.write(frame)

        # Store the count and stats for this video
        ball_detection_counts[video_name] = ball_count
        detection_ratio = detection_frames / total_frames  # Calculate the detection ratio
        
        # Store the stats for each video
        video_stats[video_name] = {
            "FPS": fps,
            "Total Frames": total_frames,
            "Detection Frames": detection_frames,
            "Detection Ratio": detection_ratio
        }

        # Release the video capture and writer for this file
        cap.release()
        output_video.release()
    
    return ball_detection_counts, video_stats,detection_count,conf

# Example Usage
video_paths = [
    '/kaggle/input/test-data-inference/Maria Sharapova has NO quit  - US Open Tennis Championships (1080p h264 youtube).mp4'
]    

# Path to save the output video
output_path = '/kaggle/working'  # Adjust the output path as needed

# Count 'sports ball' detections in each video and save the tracked video
ball_counts, video_statistics,detection_found,conf = count_and_track_sports_ball_detections(video_paths, output_path)

# Output the detection counts and stats
print("Sports ball detection counts by video format:")
for video, count in ball_counts.items():
    print(f"{video}: {count} detections")

print("\nVideo statistics:")
for video, stats in video_statistics.items():
    print(f"{video}:")
    for key, value in stats.items():
        print(f"  {key}: {value}")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to runs/detect/track
Sports ball detection counts by video format:
Maria Sharapova has NO quit  - US Open Tennis Championships (1080p h264 youtube).mp4: 495 detections

Video statistics:
Maria Sharapova has NO quit  - US Open Tennis Championships (1080p h264 youtube).mp4:
  FPS: 29.97002997002997
  Total Frames: 923
  Detection Frames: 495
  Detection Ratio: 0.5362946912242686


In [9]:
# Function to process video and count 'sports ball' detections
def count_sports_ball_detections(video_paths, conf=0.1):
    ball_detection_counts = {}

    # Iterate over each video path
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        ball_count = 0  # Counter for sports ball detections
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
        # Process the video frame by frame
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Detect objects in the frame using YOLO
            results = model(frame, conf=conf,verbose=False)
            
            # Iterate over detected objects and count sports balls
            for result in results:
                for obj in result.boxes:
                    # Check if the label is 'sports ball'
                    if 'tennis ball' in result.names[int(obj.cls)]:
                        ball_count += 1
        
        # Store the count for this video
        video_name = video_path.split('/')[-1]  # Extract video file name
        ball_detection_counts[video_name] = ball_count
     
        # Release the video capture for this file
        cap.release()
    
    return ball_detection_counts,total_frames





# List of video paths and speed factors
video_paths = ['/kaggle/input/test-data-inference/Maria Sharapova has NO quit  - US Open Tennis Championships (1080p h264 youtube).mp4',
#                '/kaggle/input/different-speed-of-tennis-data-set/4_x_input_video.mp4'
   
]

# Count 'sports ball' detections in each video
ball_counts,total_frames = count_sports_ball_detections(video_paths)
print(total_frames)
# Output the detection counts
print("Sports ball detection counts by video format:")
for video, count in ball_counts.items():
    print(f"{video}: {count} detections" )

923
Sports ball detection counts by video format:
Maria Sharapova has NO quit  - US Open Tennis Championships (1080p h264 youtube).mp4: 504 detections


In [10]:

result = model.track('/kaggle/input/test-data-inference/Maria Sharapova has NO quit  - US Open Tennis Championships (1080p h264 youtube).mp4',conf=0.1, save=True,verbose=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/923) /kaggle/input/test-data-inference/Maria Sharapova has NO quit  - US Open Tennis Championships (1080p h264 youtube).mp4: 640x384 1 tennis ball, 62.8ms
video 1/1 (frame 2/923) /kaggle/input/test-data-inference/Maria Sharapova has NO quit  - US Open Tennis Championships (1080p h264 youtube).mp4: 640x384 1 tennis ball, 62.7ms
video 1/1 (frame 3/923) /kaggle/input/test-data-inference/Maria Sharapova has NO quit  - US Open Tennis Cham

In [ ]:
df=pd.DataFrame({'conf':conf,'detect':detection_found})

In [ ]:
df_new=df[(df["detect"]==True)& (df['conf']<0.1)]
df_new

In [ ]:
import cv2

def count_and_track_sports_ball_detections(video_paths, output_path, initial_conf=0.1, iou_thresh=0.3, resize_factor=1.0, fps_factor=0.05):
    ball_detection_counts = {}

    # Iterate over each video path
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * resize_factor)
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * resize_factor)
        fps = cap.get(cv2.CAP_PROP_FPS)
        print(fps)
        # Adjust FPS for processing
        adjusted_fps = fps * fps_factor
        print(adjusted_fps)
        print()
        # Video writer to save the output video with tracking
        video_name = video_path.split('/')[-1]
        output_video = cv2.VideoWriter(f"{output_path}/{video_name}_tracked.avi", 
                                       cv2.VideoWriter_fourcc(*'XVID'), adjusted_fps, 
                                       (frame_width, frame_height))

        ball_count = 0  # Counter for sports ball detections
        no_detection_streak = 0  # Counter for consecutive frames without detections

        # Process the video frame by frame with tracking enabled
        results = model.track(video_path, conf=initial_conf, iou=iou_thresh, save=True, verbose=False)

        # Iterate over frames in the tracking result
        for frame_result in results:
            detections_found = False  # Flag to indicate detection in this frame
            frame = frame_result.orig_img  # Original frame from the result
            
            # Resize frame if a resize factor is specified
            if resize_factor != 1.0:
                frame = cv2.resize(frame, (frame_width, frame_height))

            # Iterate over detected objects in each frame
            for obj in frame_result.boxes:
                # Check if the label is 'tennis ball'
                if 'tennis ball' in frame_result.names[int(obj.cls)]:
                    ball_count += 1
                    detections_found = True  # Detection found in this frame
                    no_detection_streak = 0  # Reset streak if detection is found

                    # Get bounding box coordinates and draw on the frame
                    x1, y1, x2, y2 = map(int, obj.xyxy[0].cpu().numpy())
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, 'Tennis Ball', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    break  # Exit after first detection

            # Adjust confidence threshold if no detections found in several frames
            if not detections_found:
                no_detection_streak += 1
                if no_detection_streak >= 5:  # Lower confidence threshold after 5 frames without detections
                    initial_conf = max(0.05, initial_conf - 0.05)
            else:
                initial_conf = max(0.1, initial_conf)  # Reset to the initial confidence threshold
            
            # Write the processed frame with bounding boxes to the output video
            output_video.write(frame)

        # Store the count for this video
        ball_detection_counts[video_name] = ball_count
        
        # Release the video capture and writer for this file
        cap.release()
        output_video.release()
    
    return ball_detection_counts

video_paths = [
    '/kaggle/input/test-data/Tennis_Test_1.mp4'
]    

# Path to save the output video
output_path = '/kaggle/working'  # Adjust the output path as needed

# Count 'sports ball' detections in each video and save the tracked video
ball_counts = count_and_track_sports_ball_detections(video_paths, output_path, fps_factor=0.05)

# Output the detection counts
print("Sports ball detection counts by video format:")
for video, count in ball_counts.items():
    print(f"{video}: {count} detections")



In [ ]:
import cv2

def count_and_track_sports_ball_detections(video_paths, output_path, initial_conf=0.1, iou_thresh=0.3, resize_factor=1.0, fps_factor=0.5):
    ball_detection_counts = {}

    # Iterate over each video path
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * resize_factor)
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * resize_factor)
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Adjust FPS for processing
        adjusted_fps = fps * fps_factor

        # Video writer to save the output video with tracking
        video_name = video_path.split('/')[-1]
        output_video = cv2.VideoWriter(f"{output_path}/{video_name}_tracked.avi", 
                                       cv2.VideoWriter_fourcc(*'XVID'), adjusted_fps, 
                                       (frame_width, frame_height))

        ball_count = 0  # Counter for sports ball detections

        # Process the video frame by frame with tracking enabled
        results = model.track(video_path, conf=initial_conf, iou=iou_thresh, save=True, verbose=False)

        # Iterate over frames in the tracking result
        for frame_result in results:
            frame = frame_result.orig_img  # Original frame from the result
            
            # Resize frame if a resize factor is specified
            if resize_factor != 1.0:
                frame = cv2.resize(frame, (frame_width, frame_height))

            # Check for detections in the current frame
            for obj in frame_result.boxes:
                # Check if the label is 'tennis ball'
                if 'tennis ball' in frame_result.names[int(obj.cls)]:
                    ball_count += 1  # Increment the ball count

                    # Get bounding box coordinates and draw on the frame
                    x1, y1, x2, y2 = map(int, obj.xyxy[0].cpu().numpy())
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, 'Tennis Ball', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    break  # Exit after first detection to avoid counting the same ball multiple times

            # Write the processed frame with bounding boxes to the output video
            output_video.write(frame)

        # Store the count for this video
        ball_detection_counts[video_name] = ball_count
        
        # Release the video capture and writer for this file
        cap.release()
        output_video.release()
    
    return ball_detection_counts

video_paths = [
    '/kaggle/input/test-data/Tennis_Test_1.mp4'
]    

# Path to save the output video
output_path = '/kaggle/working'  # Adjust the output path as needed

# Count 'sports ball' detections in each video and save the tracked video
ball_counts = count_and_track_sports_ball_detections(video_paths, output_path, fps_factor=0.5)

# Output the detection counts
print("Sports ball detection counts by video format:")
for video, count in ball_counts.items():
    print(f"{video}: {count} detections")


In [ ]:
# import cv2

# def count_and_track_sports_ball_detections(video_paths, output_path, initial_conf=0.1, iou_thresh=0.3, resize_factor=1.0):
#     ball_detection_counts = {}

#     # Iterate over each video path
#     for video_path in video_paths:
#         cap = cv2.VideoCapture(video_path)
#         frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * resize_factor)
#         frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * resize_factor)
#         fps = cap.get(cv2.CAP_PROP_FPS)

#         # Video writer to save the output video with tracking
#         video_name = video_path.split('/')[-1]
#         output_video = cv2.VideoWriter(f"{output_path}/{video_name}_tracked.avi", 
#                                        cv2.VideoWriter_fourcc(*'XVID'), fps, 
#                                        (frame_width, frame_height))

#         ball_count = 0  # Counter for sports ball detections
#         no_detection_streak = 0  # Counter for consecutive frames without detections

#         # Process the video frame by frame with tracking enabled
#         results = model.track(video_path, conf=initial_conf, iou=iou_thresh, save=True, verbose=False)

#         # Iterate over frames in the tracking result
#         for frame_result in results:
#             detections_found = False  # Flag to indicate detection in this frame
#             frame = frame_result.orig_img  # Original frame from the result
            
#             # Resize frame if a resize factor is specified
#             if resize_factor != 1.0:
#                 frame = cv2.resize(frame, (frame_width, frame_height))

#             # Iterate over detected objects in each frame
#             for obj in frame_result.boxes:
#                 # Check if the label is 'tennis ball'
#                 if 'tennis ball' in frame_result.names[int(obj.cls)]:
#                     ball_count += 1
#                     detections_found = True  # Detection found in this frame
#                     no_detection_streak = 0  # Reset streak if detection is found

#                     # Get bounding box coordinates and draw on the frame
#                     x1, y1, x2, y2 = map(int, obj.xyxy[0].cpu().numpy())
#                     cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                     cv2.putText(frame, 'Tennis Ball', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
#                     break  # Exit after first detection

#             # Adjust confidence threshold if no detections found in several frames
#             if not detections_found:
#                 no_detection_streak += 1
#                 if no_detection_streak >= 5:  # Lower confidence threshold after 5 frames without detections
#                     initial_conf = max(0.05, initial_conf - 0.05)
#             else:
#                 initial_conf = max(0.1, initial_conf)  # Reset to the initial confidence threshold
            
#             # Write the processed frame with bounding boxes to the output video
#             output_video.write(frame)

#         # Store the count for this video
#         ball_detection_counts[video_name] = ball_count
        
#         # Release the video capture and writer for this file
#         cap.release()
#         output_video.release()
    
#     return ball_detection_counts

# video_paths = [
#     '/kaggle/input/test-data/Tennis_Test_1.mp4'
# ]    

# # Path to save the output video
# output_path = '/kaggle/working'  # Adjust the output path as needed

# # Count 'sports ball' detections in each video and save the tracked video
# ball_counts = count_and_track_sports_ball_detections(video_paths, output_path)

# # Output the detection counts
# print("Sports ball detection counts by video format:")
# for video, count in ball_counts.items():
#     print(f"{video}: {count} detections")


# not frame wise

In [ ]:
def count_sports_ball_detections(video_paths, initial_conf=0.1, iou_thresh=0.3, resize_factor=1.0):
    ball_detection_counts = {}

    # Iterate over each video path
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        ball_count = 0  # Counter for sports ball detections
        no_detection_streak = 0  # Counter for consecutive frames without detections
        current_conf = initial_conf  # Initialize current confidence threshold
        
        # Process the video frame by frame
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  # Break if no more frames
            
            # Resize the frame if needed
            if resize_factor != 1.0:
                frame = cv2.resize(frame, (int(frame.shape[1] * resize_factor), int(frame.shape[0] * resize_factor)))
            
            # Run model tracking on the current frame with the updated confidence threshold
            results = model.track(frame, conf=current_conf, iou=iou_thresh, save=True, verbose=False)
            detections_found = False  # Flag to indicate detection in this frame
            
            # Iterate over detected objects in the current frame
            for obj in results[0].boxes:
                # Check if the label is 'tennis ball'
                if 'tennis ball' in results[0].names[int(obj.cls)]:
                    ball_count += 1
                    detections_found = True  # Detection found in this frame
                    no_detection_streak = 0  # Reset streak if detection is found
                    current_conf = initial_conf  # Reset confidence threshold
                    break  # Break after first detection
            
            # Adjust confidence threshold if no detections found in several frames
            if not detections_found:
                no_detection_streak += 1
                if no_detection_streak >= 5:  # Lower confidence threshold after 5 frames without detections
                    current_conf = max(0.05, current_conf - 0.01)
            else:
                no_detection_streak = 0  # Reset streak on detection

        # Store the count for this video
        video_name = video_path.split('/')[-1]  # Extract video file name
        ball_detection_counts[video_name] = ball_count

        # Release the video capture for this file
        cap.release()
    
    return ball_detection_counts

# List of video paths
video_paths = [
    '/kaggle/input/different-speed-of-tennis-data-set/0.25_x_input_video.mp4',
    '/kaggle/input/different-speed-of-tennis-data-set/0.50_x_input_video.mp4',
    '/kaggle/input/different-speed-of-tennis-data-set/1_x_input_video.mp4',
    '/kaggle/input/different-speed-of-tennis-data-set/1.5_x_input_video.mp4',
    '/kaggle/input/different-speed-of-tennis-data-set/2_x_input_video.mp4',
    '/kaggle/input/different-speed-of-tennis-data-set/4_x_input_video.mp4'
]

# Count 'sports ball' detections in each video
ball_counts = count_sports_ball_detections(video_paths)

# Output the detection counts
print("Sports ball detection counts by video format:")
for video, count in ball_counts.items():
    print(f"{video}: {count} detections")


# COunting the detection


In [66]:
# Load YOLOv8 model
model = YOLO('/kaggle/input/updated-train-result-best2/best (2).pt')
def count_sports_ball_detections(video_paths, conf=0.05):
    ball_detection_counts = {}

    # Iterate over each video path
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        ball_count = 0  # Counter for sports ball detections
        
        # Process the video frame by frame
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Detect objects in the frame using YOLO
            results = model(frame, conf=conf,verbose=False) # verbose used to dont print results 
            
            # Iterate over detected objects and count sports balls
            for result in results:
                for obj in result.boxes:
                    # Check if the label is 'sports ball'
                    if 'tennis ball' in result.names[int(obj.cls)]:
                        ball_count += 1
        
        # Store the count for this video
        video_name = video_path.split('/')[-1]  # Extract video file name
        ball_detection_counts[video_name] = ball_count
        # Release the video capture for this file
        cap.release()
    
    return ball_detection_counts
# List of video paths and speed factors
video_paths = [
    # '/kaggle/input/different-speed-of-tennis-data-set/0.25_x_input_video.mp4',  # 0.25x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/0.50_x_input_video.mp4',     # 0.5x speed video
   '/kaggle/input/test-data/Tennis_Test_1.mp4', #1x speed Video
    # '/kaggle/input/different-speed-of-tennis-data-set/1.5_x_input_video.mp4', #1.5x Speed Video
    # '/kaggle/input/different-speed-of-tennis-data-set/2_x_input_video.mp4',      # 2x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/4_x_input_video.mp4'    # 4x speed Video
]    

# Count 'sports ball' detections in each video
ball_counts = count_sports_ball_detections(video_paths)

# Output the detection counts
print("Tennis ball detection counts by video format:")
for video, count in ball_counts.items():
    print(f"{video}: {count} detections")

Tennis ball detection counts by video format:
Tennis_Test_1.mp4: 415 detections


In [ ]:
video_1='/kaggle/input/input-video-1x/Federer vs Nadal .mp4'
result = model.track(video_1,conf=0.05, save=True,verbose=False)

In [ ]:
import cv2

# Load YOLOv8 model
model = YOLO('/kaggle/input/updated-result-3rd-training/last (2).pt')

def count_sports_ball_detections(video_paths, conf=0.1, frame_skip_factors=None):
    ball_detection_counts = {}

    if frame_skip_factors is None:
        # Default frame skip factors for different video speeds (can be customized)
        frame_skip_factors = {
            # '0.25x': 8,   # Process every 8th frame for slow-motion videos
            # '0.50x': 4,   # Process every 4th frame for slower videos
            '1x': 1,      # Process every frame for real-time speed
        #     '1.5x': 1,    # Process every frame for faster videos
        #     '2x': 1,      # Process every frame
        #     '4x': 0.5     # Process every frame, may need additional processing to avoid missed detections
        }

    # Iterate over each video path
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        ball_count = 0  # Counter for sports ball detections
        frame_id = 0  # To keep track of frame count
        video_name = video_path.split('/')[-1]  # Extract video file name

        # Identify video speed based on its name
        speed_label = video_name.split('_')[0]  # Extract the speed (e.g., '0.25x')
        frame_skip_factor = frame_skip_factors.get(speed_label, 1)  # Get corresponding frame skip factor

        # Process the video frame by frame
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Only process the frame if it's not skipped
            if frame_id % frame_skip_factor == 0:
                # Detect objects in the frame using YOLO
                results = model(frame, conf=conf, verbose=False)
                
                # Iterate over detected objects and count tennis balls
                for result in results:
                    for obj in result.boxes:
                        # Check if the label is 'tennis ball'
                        if 'tennis ball' in result.names[int(obj.cls)]:
                            ball_count += 1
            
            frame_id += 1

        # Store the count for this video
        ball_detection_counts[video_name] = ball_count

        # Release the video capture for this file
        cap.release()
    
    return ball_detection_counts

# List of video paths and speed factors
video_paths = [
    # '/kaggle/input/different-speed-of-tennis-data-set/0.25_x_input_video.mp4',  # 0.25x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/0.50_x_input_video.mp4',  # 0.5x speed video
    '/kaggle/input/different-speed-of-tennis-data-set/1_x_input_video.mp4',     # 1x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/1.5_x_input_video.mp4',   # 1.5x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/2_x_input_video.mp4',     # 2x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/4_x_input_video.mp4'      # 4x speed video
]

# Count 'sports ball' detections in each video
ball_counts = count_sports_ball_detections(video_paths)

# Output the detection counts
print("Sports ball detection counts by video format:")
for video, count in ball_counts.items():
    print(f"{video}: {count} detections")
